# 빅데이터 분석기사 실기 준비
## 프리렉 출판사 빅데이터 분석기사 실기 책 자료 참고

## 데이터 탐색

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/mtcars.csv')
print(df.head())

mpg : 연비
Unnamed : 0 : 차 모델 명
cyl : 엔진 기통
disp : 배기량
hp : 마력
drat : 뒤 차축비
wt : 무게
qsec : 1/4 mile 도달 시간
vs : V형 엔진/ 직렬엔진
am : 변속기 
gear : 전진기어 개수
carb : 기화기 개수

In [ ]:
print(df.shape)

In [ ]:
print(type(df))

In [ ]:
print(df.columns)

In [ ]:
print(df.describe())

cyl - 실린더가 50기통임은 이상치로 여겨짐

In [ ]:
print(df['am'].unique())
print(df['gear'].unique())
print(df['vs'].unique())

In [ ]:
print(df.info())

In [ ]:
print(df.corr())

종속 변수와 독립 변수 구분 \
mpg를 종속변수로, 외 열은 독립 변수

In [ ]:
x = df.drop(columns = 'mpg')
y = df['mpg']

In [ ]:
print(x.head())
print(x.columns)

In [ ]:
print(y.head())

## 데이터 전처리

In [ ]:
# 불필요한 컬럼 처리
print(x.head())
x = x.iloc[:,1:]

In [ ]:
print(x.head())

In [ ]:
# 결측치 처리
x.isnull().sum()

In [ ]:
# cyl, qsec 에만 존재 확인, 평균값으로 대치
x_cyl_mean = x['cyl'].mean()

print(x_cyl_mean)

In [ ]:
x['cyl'] = x['cyl'].fillna(x_cyl_mean)

In [ ]:
# qsec는 중위값으로 대치
x_qsec_median = x['qsec'].median()
print(x_qsec_median)

In [ ]:
x['qsec'] = x['qsec'].fillna(x_qsec_median)

In [ ]:
x.isnull().sum()

In [ ]:
# gear에 잘못된 값 바꾸기
print(x['gear'].unique())

In [ ]:
x['gear'] = x['gear'].replace('*3', '3').replace('*5','5')

In [ ]:
print(x['gear'].unique())

In [ ]:
# 이상값 처리, IQR 처리

# 기초통계량 확인
x_describe = x.describe()
print(x_describe)

In [ ]:
print(x_describe.loc['75%'], x_describe.loc['25%'])

In [ ]:
x_iqr = x_describe.loc['75%'] - x_describe.loc['25%']
print(x_iqr)

In [ ]:
print(x_describe.loc['75%'] + (1.5 * x_iqr))

In [ ]:
print(x_describe.loc['max'])

In [ ]:
#cyl, hp, wt, qsec, carb 는 이상치 존재
# 이외에 cyl은 14 이상이 최대이기에 초과 값은 이상치 처리

# cyl
print(x.loc[x['cyl'] > 14])
x.loc[14,'cyl'] = 14

In [ ]:
# hp
print(x.loc[x['hp'] > 305.25])
x.loc[30,'hp'] = 305.25

In [ ]:
# 최소 경계
print(x_describe.loc['25%'] - (1.5 * x_iqr))

In [ ]:
x_describe.loc['min']

In [ ]:
#표준편차 활용 이상치 처리
#평균 +- 1.5표준편차 범위 밖은 이상치
#함수로 이상치 인덱스 출력
def outlier(data, column):
  mean = data[column].mean()
  std = data[column].std()
  lowest = mean - (1.5 * std)
  highest = mean + (1.5 * std)
  print('lowest : ', lowest, '\n', 'highest : ', highest)
  outlier_index = data[column][(df[column] < lowest) | (data[column] > highest)].index
  return outlier_index

In [ ]:
outlier(x, 'qsec')

In [ ]:
print(x.loc[24,'qsec'])

In [ ]:
x.loc[24,'qsec'] = 42.245
print(x.loc[24,'qsec'])

In [ ]:
print(outlier(x, 'carb'))

In [ ]:
print(x.loc[[29,30], 'carb'])

In [ ]:
x.loc[[29,30], 'carb'] = 5.235

In [ ]:
print(x.loc[[29,30], 'carb'])

## 데이터 스케일링

독립변수 범위를 동일하게 만들어줌으로 머신러닝 결과 왜곡을 막고 특정 독립 변수를 무지하지 않기 위해

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
temp = x[['qsec']]

In [ ]:
scaler = StandardScaler()

In [ ]:
print(scaler.fit_transform(temp))

In [ ]:
qsec_s_scaler = pd.DataFrame(scaler.fit_transform(temp))

In [ ]:
print(qsec_s_scaler.describe())

평균이 0에 근사하고 표준편차는 1에 근사한 정규 분표 형태로 변환되었음

In [ ]:
#최소최대 크기 변환
from sklearn.preprocessing import MinMaxScaler

In [ ]:
temp = x[['qsec']]

In [ ]:
scaler = MinMaxScaler()

In [ ]:
qsec_m_scaler = pd.DataFrame(scaler.fit_transform(temp))

In [ ]:
print(qsec_m_scaler)

In [ ]:
# 로버스트 크기 변환
from sklearn.preprocessing import RobustScaler

temp = x[['qsec']]

In [ ]:
scaler = RobustScaler()

qsec_r_scaler = pd.DataFrame(scaler.fit_transform(temp))

print(qsec_r_scaler)

In [ ]:
print(qsec_r_scaler.describe())

### 데이터 타입 변경

In [ ]:
print(x.info())

In [ ]:
# gear는 수치형이 들어가는 연속형 변수이므로 형 변환
x['gear'] = x['gear'].astype(int)

In [ ]:
x['gear'].dtype

In [ ]:
# 범주형을 수치형으로 인코딩
# 원핫인코딩 pd.get_dummies()로 수행
print(pd.get_dummies(x['am']))


In [ ]:
# 이는 여러 단점으로 불편하니 새로운 열이 늘어나지 않고 1개의 열로 판단 되게 수정
# data 종류가 n개라면 n-1개 열만 새로 생성 , get_dummies(drop_first = True) 로
print(pd.get_dummies(x['am'], drop_first = True))

In [ ]:
# get_dummies() 는 기본적으로 범주형 데이터를 골라 인코딩 처리
# sklearn 에서 OneHotEncoder()도 비슷한 기능을 하지만 전체 변수에 인코딩하는 특징
print(x.info())
print(pd.get_dummies(x, drop_first=True))

In [ ]:
# 라벨 인코딩 - 일련번호 부여 방식으로 점차 숫자 커짐 -> 이 때문에 트리 계열에서만 주로 사용

print(x['am'].head())

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

print(encoder.fit_transform(x['am']))
#am 데이터가 2개라 원핫과 비슷하게 변화

In [ ]:
# 3개 이상에서의 변화를 보기 위한 예시
fruit = ['apple', 'banana', 'grape']
fruit_new = encoder.fit_transform(fruit)
print(fruit, fruit_new)

In [ ]:
# 수동 인코딩 replace
x['am_new'] = x['am'].replace('manual', 0).replace('auto', 1)
print(x.head())

In [ ]:
x = x.drop(columns = 'am')
print(x.head())

In [ ]:
# 파생 변수 만들기
# wt 를 등급 나누기, 평균인 3.3보다 낮으면 0 아니면 1
print(x['wt'] < 3.3)
condition = x['wt'] < 3.3

In [ ]:
x.loc[condition, 'wt_class'] = 0
x.loc[~condition, 'wt_class'] = 1


In [ ]:
print(x[['wt','wt_class']])

In [ ]:
x = x.drop(columns = ['wt'])

In [ ]:
print(x.head())

In [ ]:
# qsec_4 만들기, qsec의 4배로 1mile 도달 시간 변수
x['qsec_4'] = x['qsec'] * 4

In [ ]:
print(x[['qsec', 'qsec_4']])

In [ ]:
x = x.drop(columns = ['qsec'])
print(x.head())

## 학습 데이터로 공부하기
### 모델 생성과 모델 검증

In [ ]:
# 데이터 분리 위한 train_Test_split 함수
from sklearn.model_selection import train_test_split

In [ ]:
#train_test_split에는 첫째 독립변수 데이터 셋, 둘째 종속변수 데이터 셋, 셋째 테스트 데이터로 분리할 비율값을 입력
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, \
                                                    random_state = 10)

In [ ]:
print(x_train.head())

In [ ]:
print(y_train.head())

In [ ]:
print(x_test.head())

In [ ]:
print(y_test.head())

### 모델링(학습에 대한 평가)

#### 종속변수가 연속형 변수인 mpg, 범주형 변수인 am 두 가지 연습

In [ ]:
# 선형회귀
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(x_train, y_train)
y_train_predicted = model.predict(x_train)
y_test_predicted = model.predict(x_test)


In [ ]:
#y절편
print(model.intercept_)

In [ ]:
#독립변수들의 각 기울기 
print(model.coef_)

In [ ]:
# 예측모델의 정확성
# 결정계수
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))


과적합 현상 나타남

In [ ]:
#결정계수 계산하는 r2_Score
from sklearn.metrics import r2_score

#MAE 계산하는 mean_absolute_error
from sklearn.metrics import mean_absolute_error

#MSE
from sklearn.metrics import mean_squared_error

#제곱근 계산 위한 numpy
import numpy as np

In [ ]:
print(r2_score(y_train, y_train_predicted))
print(r2_score(y_test, y_test_predicted))
print(mean_squared_error(y_test, y_test_predicted))
print(np.sqrt(mean_squared_error(y_test, y_test_predicted)))
print(mean_absolute_error(y_test, y_test_predicted))


In [ ]:
# 램덤포레스트 회귀
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state = 10)

model.fit(x_train, y_train)

y_train_predicted = model.predict(x_train)

y_test_predicted = model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print(r2_score(y_train, y_train_predicted))

print(r2_score(y_test, y_test_predicted))

print(mean_squared_error(y_test, y_test_predicted))
print(mean_absolute_error(y_test, y_test_predicted))

In [ ]:
# 트리의 성능을 올리기 위해
# 트리 개수, 기준을 변경하고 지표는 MSE->MAE로

model = RandomForestRegressor(n_estimators = 1000, criterion = 'mae', random_state = 10)

model.fit(x_train, y_train)
y_train_predicted = model.predict(x_train)
y_test_predicted = model.predict(x_test)

In [ ]:
print(r2_score(y_train, y_train_predicted))
print(r2_score(y_test, y_test_predicted))

In [ ]:
print(mean_squared_error(y_test, y_test_predicted))
print(mean_absolute_error(y_test, y_test_predicted))

In [ ]:
# 그레디언트 부스팅 회귀
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state = 10)

model.fit(x_train, y_train)

y_train_predicted = model.predict(x_train)

y_test_predicted = model.predict(x_test)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print(r2_score(y_train, y_train_predicted))

print(r2_score(y_test, y_test_predicted))

print(mean_squared_error(y_test, y_test_predicted))
print(mean_absolute_error(y_test, y_test_predicted))

In [ ]:
# 익스트림 그레디언트 부스팅 회귀(XGB Regressor)
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror', random_state = 10)

model.fit(x_train, y_train)

y_train_predicted = model.predict(x_train)

y_test_predicted = model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print(r2_score(y_train, y_train_predicted))

print(r2_score(y_test, y_test_predicted))

print(mean_squared_error(y_test, y_test_predicted))
print(mean_absolute_error(y_test, y_test_predicted))

In [ ]:
# 분류 모델링
# 기존 데이터셋을 가공해 수행
# 쫑속변수를 변속기 유형인 am_new로 설쩡하기 위해

# xtrain 에서 종속 변수 삭제하기
x_train2 = x_train.drop(columns = 'am_new')

y_train2 = x_train['am_new']

x_test2 = x_test.drop(columns = 'am_new')

y_test2 = x_test['am_new']

In [ ]:
print(x_train2)
print(y_train2)

In [ ]:
# 의사결정나무 분류
# 과적합 가능성이 큰 모델
# 특성들이 단독으로 처리되어 데이터 스케일 영향을 덜 받고 예측, 분류에 모두 사용 가능
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

model.fit(x_train2, y_train2)
y_test2_predicted = model.predict(x_test2)

In [ ]:
#ROC-AUC 계산
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [ ]:
print(roc_auc_score(y_test2, y_test2_predicted))

print(accuracy_score(y_test2, y_test2_predicted))

print(precision_score(y_test2, y_test2_predicted))

print(recall_score(y_test2, y_test2_predicted))



In [ ]:
# 랜덤포레스트
# 앙상블 모델로 다수 트리 중 투표
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train2, y_train2)

y_test2_predicted = model.predict(x_test2)

In [ ]:
print(roc_auc_score(y_test2, y_test2_predicted))


In [ ]:
# 로지스틱 회귀

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train2, y_train2)

y_test2_predicted = model.predict(x_test2)

In [ ]:
print(roc_auc_score(y_test2, y_test2_predicted))


In [ ]:
# XGB 분류
# 다수의 약한 분류기를 모아 정확도를 향상, 병렬처리
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(x_train2, y_train2)

y_test2_predicted = model.predict(x_test2)

In [ ]:
print(y_test2_predicted)

In [ ]:
# 0,1로 분류되는 종속변수가 각 값엥 대해 어느 정도 확률은 가지는지
# 1열은 0으로 2열은 1로 분류될 확률
# predict_proba() 사용

y_test2_proba = model.predict_proba(x_test2)
print(y_test2_proba)

In [ ]:
print(roc_auc_score(y_test2, y_test2_predicted))


## 작업 1유형 단순한 데이터 처리

사용 데이터는 Boston

문제 : boston 데이터 셋의 MEDV 칼럼에 대해 가장 작은 값부터 순서대로 10개 행을 출력 \
즉, 오름차순으로 정렬된 MEDV값에서 Top 10을 구하시오


In [ ]:
# pandas 가져오고 csv불러오기
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

In [ ]:
print(data.head())

In [ ]:
print(data.sort_values(by = 'MEDV', ascending = True))

In [ ]:
print(data.sort_values(by = 'MEDV', ascending = True)['MEDV'])

In [ ]:
print(data.sort_values(by = 'MEDV', ascending = True)['MEDV'].head(10))

문제 : boston 데이터셋의 RM 데이터 결측치 처리를 평균값으로 대치하거나 삭제. \
평균으로 대치하고 산출된 표준편차 값과 결측치 삭제한 후 산출된 표준편차 값의 차이를 구하라

In [ ]:
print(data.head())

In [ ]:
print(data['RM'].isnull().sum())

In [ ]:
print(data['RM'].mean())
#비교를 위해 평균 대치의 DF
data_mean = data['RM'].copy()

In [ ]:
print(data_mean.isnull().sum())

In [ ]:
print(data_mean.head(3))

In [ ]:
rm_mean = data_mean.mean()
print(rm_mean)

In [ ]:
print(data_mean.fillna(rm_mean, inplace = False))

In [ ]:
print(data_mean.fillna(rm_mean, inplace = True))
print(data_mean.isnull().sum())

In [ ]:
# del 경우 
data_del = data['RM'].copy()

In [ ]:
print(data_del.isnull().sum())

In [ ]:
print(data_del.shape)

In [ ]:
data_del.dropna(inplace = True)

In [ ]:
print(data_del.shape)

In [ ]:
print(data_del.isnull().sum())

In [ ]:
print(data_mean.std())

In [ ]:
print(data_del.std())

In [ ]:
print(abs(data_mean.std() - data_del.std()))

문제 \
이상값 확인. 데이터 셋의 ZN 칼럼을 대상으로 ZN 값의 평균값에서 표준 편차의 1.5배보다 크거나 작은 zn값의 합계

In [ ]:
print(data['ZN'].head())

In [ ]:
print(data['ZN'].mean())
zn_mean = data['ZN'].mean()

In [ ]:
print(data['ZN'].std())
zn_std = data['ZN'].std()

In [ ]:
zn_max = zn_mean + (1.5 * zn_std)

In [ ]:
print(zn_max)

In [ ]:
zn_min = zn_mean - (1.5 * zn_std)

In [ ]:
print(zn_min)

In [ ]:
print(data[data['ZN'] > zn_max]['ZN'])
zn_max2 = data[data['ZN'] > zn_max]['ZN']

In [ ]:
print(data[data['ZN'] < zn_min])
print(sum(zn_max2))

문제 \
CHAS 칼럼과 RAD 칼럼을 제외한 칼럼에 한해 칼럼별 IQR \
단, 출력 구조는 2열이고 1열은 보스턴 데이터 셋의 칼럼 이름이 표시되어야


In [ ]:
print(data.head())

In [ ]:
print(data.drop(columns = ['CHAS', 'RAD']))

In [ ]:
data_col12 = data.drop(columns = ['CHAS', 'RAD'])
print(data_col12.shape)

In [ ]:
data_col12_desc = data_col12.describe()

In [ ]:
print(data_col12_desc)

In [ ]:
print(data_col12_desc.iloc[[4,6]])

In [ ]:
print(data_col12_desc.iloc[[4,6]].T)
data_col12_Desc_T = data_col12_desc.iloc[[4,6]].T

In [ ]:
print(data_col12_Desc_T['75%'] - data_col12_Desc_T['25%'])

문제 \
순위 구하기 \
MEDV 칼럼의 30번째로 큰 값을 구합니다. 그리고 1번째부터 29번째로 큰 값을 30번째 큰 값으로 변경.
이후 MEDV 칼럼의 평균, 중위, 최소, 최대를 한 줄에 출력

In [ ]:
print(data['MEDV'])

In [ ]:
data_new = data['MEDV'].sort_values(ascending = False)

print(data_new.head(30))

In [ ]:
print(data_new.iloc[29])

In [ ]:
data_new.iloc[0:29] = data_new.iloc[29]

In [ ]:
print(data_new.mean(), data_new.median(), data_new.min(), data_new.max())

문제 \
그룹별 집계/요약 \
TAX 칼럼이 TAX 칼럼의 중위값보다 큰 데이터를 대상으로 , CHAS, RAD 칼럼 순으로 그룹 지은 후 각 그룹의 데이터 개수 \
단, 두 칼럼 데이터 개수는 count 라는 칼럼으로 출력

In [ ]:
data[data['TAX'] > data['TAX'].median()].groupby(['CHAS','RAD'])['RAD'].count()

In [ ]:
data_new3 = data[data['TAX'] > data['TAX'].median()].groupby(['CHAS','RAD'])['RAD'].count()
data_new3 = pd.DataFrame(data_new3)
print(data_new3)

In [ ]:
data_new3.columns = ['COUNT']
print(type(data_new3))
print(data_new3)

문제 \
오름차순,내림차순 정렬 \ 
TAX 기준 오름차순으로 정렬 - 내림차순으로 정렬 만들기 \
동일한 위치에 있는 데이터의 차이에서 분산 계산

In [ ]:
tax_asc = data['TAX'].sort_values(ascending = True)
tax_desc = data['TAX'].sort_values(ascending = False)

In [ ]:
tax_asc.reset_index(drop = True, inplace = True)
tax_desc.reset_index(drop = True, inplace = True)


In [ ]:
data_concat = pd.concat([tax_asc, tax_desc], axis = 1)
data_concat

In [ ]:
data_concat['diff'] = abs(data_concat.iloc[:, 0] - data_concat.iloc[:, 1])

In [ ]:
data_concat

In [ ]:
data_concat['diff'].var()

문제 \
최소최대 변환 \ 
MEDV 를 최소최대로 변환하고 0.5보다 큰 값을 가지는 레코드 수 구하기

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
data_minmax = scaler.fit_transform(data)

In [ ]:
print(type(data_minmax))

In [ ]:
data_minmax = pd.DataFrame(data_minmax, columns = data.columns)

In [ ]:
data_minmax['MEDV'].describe()

In [ ]:
data_minmax[data_minmax['MEDV'] > 0.5]['MEDV'].count()

문제 \
빈도값 구하기 \
AGE 칼럼을 소수점 첫 자리에서 반올림하고, 가장 많은 비중 차지하는 AGE 값과 그 개수를 차례로 출력 \
즉, AGE 칼럼의 최빈값과 그 개수를 출력

In [ ]:
age_round = round(data['AGE'])

In [ ]:
age_round = pd.DataFrame(age_round.value_counts())
age_round

In [ ]:
age_round.iloc[0,:]

In [ ]:
# 교재 속 방법
# 1 일반적 방법ㅂ
data2 = round(data['AGE'], 0)

In [ ]:
data2 = pd.DataFrame(data2)

In [ ]:
data2.groupby(['AGE'])['AGE'].count()

In [ ]:
data3 = data2.groupby(['AGE'])['AGE'].count()
type(data3)
data3 = pd.DataFrame(data3)

In [ ]:
data3.columns = ['COUNT']

In [ ]:
data3.reset_index(drop= False, inplace = True)

In [ ]:
data3.head(3)

In [ ]:
data3.sort_values(by = 'COUNT', ascending = False, inplace = True)

In [ ]:
print(data3.iloc[0,0], data3.iloc[0,1])

In [ ]:
# 2 scipy, mode 
from scipy.stats import mode
mode(data2)

In [ ]:
int(mode(data2)[0])

In [ ]:
int(mode(data2)[1])

In [ ]:
print(int(mode(data2)[0]), int(mode(data2)[1]))

문제 \
표준 변환하기 \
DIS 칼럼을 표준화 척도로 변환해 0.4보다 크고 0.6보다 작은 값에 대한 평균 구하기 \
단, 소수점 셋째 자리에서 반올림

In [ ]:
data['DIS']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sclaer = StandardScaler()

In [ ]:
data_std = scaler.fit_transform(data)

In [ ]:
data_std = pd.DataFrame(data_std, columns = data.columns)

In [ ]:
data_std = data_std[(data_std['DIS']>0.4) & (data_std['DIS'] < 0.6)]

In [ ]:
data_std['DIS'].mean()

In [ ]:
print(round(data_std['DIS'].mean(), 2))

문제 \
유니크한 값 구하기 \
전체 칼럼에 대해 중복 제거한 unique 구하고 칼럼 별로 유니크한 값 개수를 기준으로 평균 값

In [ ]:
data_col = data.columns

In [ ]:
a = []
for i in data_col:
  a.append(data[i].nunique())

In [ ]:
a

In [ ]:
b = []
for i in a:
  b.append(int(i))

In [ ]:
b

In [ ]:
sum(b)/len(b)

## 작업 2유형 데이터 분석, 분류 모델

y_train : 고객 성별 데이터
x_train, x_test : 고객 상품 구매 속성

문제 \
성별 예측 모형 만들고 이를 평가용 데이터에 적용해 성별 예측값(남자일 확률) \
성능은 ROC-AUC로 

In [ ]:
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/x_train.csv', encoding = 'cp949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/x_test.csv', encoding = 'cp949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/y_train.csv', encoding = 'cp949')

In [ ]:
x_train.head()

In [ ]:
y_train.head()

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)


In [ ]:
x_train.info()

In [ ]:
x_train.describe().T

스케일링 필요

In [ ]:
# 불필요컬럼 삭제
x_test_cust_id = x_test['cust_id']

In [ ]:
x_train = x_train.drop(columns = 'cust_id')
x_test = x_test.drop(columns = 'cust_id')
y_train = y_train.drop(columns = 'cust_id')

In [ ]:
# 환불금액 결측치 0으로
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

In [ ]:
x_train['환불금액'].isnull().sum()

In [ ]:
# 범주형 변수 인코딩
x_train['주구매상품'].unique()

In [ ]:
x_train['주구매지점'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [ ]:
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])

print(x_train['주구매상품'].head(10))

In [ ]:
print(encoder.classes_)

In [ ]:
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])


In [ ]:
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])

print(x_train['주구매지점'].head(10))


In [ ]:
encoder.classes_

In [ ]:
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])


In [ ]:
#파생변수 만들기, 0보다 크면1, 0과 같으면 0
condition = x_train['환불금액'] > 0

In [ ]:
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0


In [ ]:
x_train[['환불금액', '환불금액_new']]

In [ ]:
x_train = x_train.drop(columns = '환불금액')

In [ ]:
x_train

In [ ]:
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0

x_test = x_test.drop(columns = '환불금액')

In [ ]:
x_train.describe().T

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)

In [ ]:
x_train

In [ ]:
x_train.describe().T

In [ ]:
x_test = pd.DataFrame(scaler.transform(x_test), columns = x_test.columns)

In [ ]:
# 상관관계 
x_train[['총구매액', '최대구매액', '환불금액_new']].corr()


In [ ]:
#0.6이상은 상관관계 있다고 판단
#다중공선성 해소를 위해 하나는 삭제
x_train = x_train.drop(columns = '최대구매액')
x_test = x_test.drop(columns = '최대구매액')


In [ ]:
# 학습
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(x_train, y_train)

y_test_predicted = model.predict(x_test)

pd.DataFrame(y_test_predicted).head(3)

In [ ]:
y_test_predicted

In [ ]:
# 학습
model = DecisionTreeClassifier(max_depth = 10, criterion = 'entropy', random_state = 10)

model.fit(x_train, y_train)

y_test_predicted = model.predict(x_test)

pd.DataFrame(y_test_predicted).head(3)

In [ ]:
y_test_proba = model.predict_proba(x_test)
pd.DataFrame(y_test_proba).head()

In [ ]:
result = pd.DataFrame(y_test_proba)[1]

In [ ]:
# 평가하기
y_train_predicted = model.predict(x_train)

from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_train_predicted)

In [ ]:
# 결과

pd.concat([x_test_cust_id, result], axis = 1)

In [ ]:
pd.concat([x_test_cust_id, result], axis = 1).rename(columns = {1 : 'gender'})

In [ ]:
final = pd.concat([x_test_cust_id, result], axis = 1).rename(columns = {1 : 'gender'})
#final.to_csv()